In [ ]:
import matplotlib
from gamtools import segregation, matrix
import os
import pandas as pd
import itertools
from sqlite3 import OperationalError
from mirnylib.numutils import removeDiagonals, observedOverExpected, fillDiagonal, PCA
import cPickle
from scipy.stats import pearsonr
import warnings
rcParams['font.size'] = 14

In [ ]:
def remove_zeros(a):
    s = np.nansum(a, axis=0) > 0
    b = a[:, s]
    c = b[s, :]
    return c, s

In [ ]:
def replace_zeros_1d(a, s, value=np.NAN):
    new_a = np.ones_like(s, dtype=a.dtype) * value
    new_a[s] = a
    return new_a

In [ ]:
def replace_zeros(a, s, value=np.NAN):
    N = len(s)
    new_a = np.ones((N, N), dtype=a.dtype) * value
    tmp = np.ones((N, len(a)), dtype=a.dtype) * value
    tmp[s, :] = a
    new_a[:, s] = tmp
    return new_a

In [ ]:
def no_na(x, y):
    assert len(x) == len(y)
    return map(np.array, zip(*[ (x,y) for x,y in zip(x, y) if np.isfinite(x) and np.isfinite(y) ]))

In [ ]:
with open('/data/pombo/rob/gam_figures/mm9_gc_1Mb.pickle', 'r') as gcfile:
    gcdict = cPickle.load(gcfile)

In [ ]:
def get_obs_over_exp(A):
        
    removeDiagonals(A, 1)
    A, mask = remove_zeros(A)
    
    M = len(A.flat)
    toclip = 100 * min(0.999, (M - 10.) / M)
    
    A = observedOverExpected(A)
    A = np.clip(A, -1e10, np.percentile(A, toclip))
    
    return A, mask

In [ ]:
def do_PCA(A, n=3):
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        A, mask = get_obs_over_exp(A)
    
    for i in [-1, 0, 1]:
        fillDiagonal(A, 1, i)
    
    A = np.corrcoef(A)
        
    PCs = PCA(A, n)[0]
                    
    return PCs, mask

In [ ]:
def get_best_PC_index(pcs, mask, gc_content):
    
    pcs_out = []
    
    top_corr = 0
    top_index = 0
    
    for i, pc in enumerate(pcs):
                
        replaced_pc = replace_zeros_1d(pc, mask)
        
        corr_this_pc = pearsonr(*no_na(replaced_pc, gc_content))[0]
        
        if corr_this_pc < 0:
            replaced_pc = 0. - replaced_pc
            corr_this_pc = abs(corr_this_pc)
            
        pcs_out.append(replaced_pc)
        
        if corr_this_pc > top_corr:
            top_corr = corr_this_pc
            top_index = i
                        
    return top_index, pcs_out

In [ ]:
hic_path = '/data/pombo/rob/mESC_1Mb_AfterICE/mESC.afterICE.chr{0}_chr{0}.1000000.my5c.txt'
new_gam_path = '/data/pombo/rob/projects/gam-follow-up-paper/data/processed/new_dataset_thrsh20/segregation_at_1Mb.chr{0}_dprime.txt.gz'
mesc400_path = '/data/pombo/gam/mesc-400/mm9/1Mb/dprime/chr{0}_chr{0}.1Mb.txt.gz'
comb_gam_path = '/data/pombo/rob/projects/gam-follow-up-paper/data/processed/combined_dataset/1Mb/segregation_at_1Mb.chr{0}_dprime.txt.gz'

In [ ]:
def file_open(c, base_path):
    (w1, w2), mat = matrix.read_file(base_path.format(c))
    return mat

def get_windows(c):
    (w1, w2), mat = matrix.read_file(gam_path.format(c))
    return w1

In [ ]:
def get_overlap(x, y):
    return float(np.logical_or(np.logical_and(x > 0, y > 0),np.logical_and(x < 0, y < 0)).sum()) / len(x)

In [ ]:
def do_pca(base_path, n_pc):
    pca_per_chrom = []
    pca_indices = []
    
    for c in range(1,20):
        
        gc_content = gcdict[c-1]
    
        data = file_open(c, base_path)

        pc, mask = do_PCA(data, n_pc)

        top_i, pc = get_best_PC_index(pc, mask, gc_content)
        
        pca_per_chrom.append(pc)

        pca_indices.append(top_i)
        
    return pca_per_chrom, pca_indices

In [ ]:
mesc_400_pc = do_pca(mesc400_path, 3)
new_gam_pc = do_pca(new_gam_path, 3)
combined_gam_pc = do_pca(comb_gam_path, 3)
hic_pc = do_pca(hic_path, 3)

In [ ]:
seg_1mb = segregation.open_segregation(
    '/data/pombo/rob/projects/gam-follow-up-paper/data/processed/combined_dataset/1Mb/segregation_at_1Mb.multibam')

In [ ]:
pca_df = seg_1mb.copy()
pca_df['gam_pca'] = np.NaN
pca_df['hic_pca'] = np.NaN

pca_df = pca_df.iloc[:,-2:]
pca_df.head()

In [ ]:
for c, (chrom_pc, i) in enumerate(zip(*combined_gam_pc), 1):
    chrom = 'chr{}'.format(c)
    is_on_chrom = pca_df.index.get_level_values(0) == chrom
    print chrom, chrom_pc[i].shape, pca_df.loc[is_on_chrom].shape
    pca_df.ix[is_on_chrom, 'gam_pca'] = chrom_pc[i]

In [ ]:
for c, (chrom_pc, i) in enumerate(zip(*hic_pc), 1):
    chrom = 'chr{}'.format(c)
    is_on_chrom = pca_df.index.get_level_values(0) == chrom
    print chrom, chrom_pc[i].shape, pca_df.loc[is_on_chrom].shape
    pca_df.ix[is_on_chrom, 'hic_pca'] = chrom_pc[i]

In [ ]:
pca_df[['gam_pca']].to_csv(
    '/data/pombo/rob/projects/gam-follow-up-paper/data/processed/combined_dataset/PCA_at_1mb.bedgraph', sep='\t')

In [ ]:
pca_df[['hic_pca']].to_csv(
    '/data/pombo/rob/projects/gam-follow-up-paper/data/processed/Dixon_PCA_at_1mb.bedgraph', sep='\t')



In [ ]:
!head head /data/pombo/rob/projects/gam-follow-up-paper/data/processed/combined_dataset/PCA_at_1mb.bedgraph

In [ ]:
!head /data/pombo/rob/projects/gam-follow-up-paper/data/processed/Dixon_PCA_at_1mb.bedgraph

In [ ]:
def neither_nan(x, y):
    x_finite = np.isfinite(x)
    y_finite = np.isfinite(y)
    both_finite = np.logical_and(x_finite, y_finite)
    return x[both_finite], y[both_finite]

In [ ]:
plt.figure(figsize=(10,8), facecolor='white')

x = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*mesc_400_pc)])
y = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*hic_pc)])
x, y = neither_nan(x,y) 
plt.hexbin(x, y, gridsize=20)
sm = np.logical_or(np.logical_and(x > np.median(x), y > 0),np.logical_and(x < 0, y < 0)).sum()
print 'Total 1Mb bins: {0}'.format(len(x))
print '1Mb bins with same compartment in both mESC-400 and Dixon Hi-X data: {0}'.format(sm)
print '% 1Mb bins with shared compartment: {0:.1%}'.format(float(sm) / len(x))
print 'Pearson corr = {}'.format(pearsonr(x, y)[0])
print pd.crosstab((x>0),(y>0))

plt.xlabel('mESC-400 PCA')
plt.ylabel('Dixon Hi-C PCA')

In [ ]:
plt.figure(figsize=(10,8), facecolor='white')

x = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*mesc_400_pc)])
y = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*new_gam_pc)])
x, y = neither_nan(x,y) 
plt.hexbin(x, y, gridsize=20)
sm = np.logical_or(np.logical_and(x > 0, y > 0),np.logical_and(x < 0, y < 0)).sum()
print 'Total 1Mb bins: {0}'.format(len(x))
print '1Mb bins with same compartment in both mESC-400 and new GAM data: {0}'.format(sm)
print '% 1Mb bins with shared compartment: {0:.1%}'.format(float(sm) / len(x))
print 'Pearson corr = {}'.format(pearsonr(x, y)[0])
print pd.crosstab((x>0),(y>0))

plt.xlabel('400x1NPs (mESC-400)')
plt.ylabel('150x4NPs')


In [ ]:
x = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*new_gam_pc)])
y = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*hic_pc)])
x, y = neither_nan(x,y) 
plt.hexbin(x, y, gridsize=20)
sm = np.logical_or(np.logical_and(x > np.median(x), y > 0),np.logical_and(x < 0, y < 0)).sum()
print 'Total 1Mb bins: {0}'.format(len(x))
print '1Mb bins with same compartment in both Dixon Hi-C and new GAM data: {0}'.format(sm)
print '% 1Mb bins with shared compartment: {0:.1%}'.format(float(sm) / len(x))
print 'Pearson corr = {}'.format(pearsonr(x, y)[0])
print pd.crosstab((x>0),(y>0))

In [ ]:
plt.figure(figsize=(10,8), facecolor='white')

x = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*combined_gam_pc)])
y = np.concatenate([chrom_pc[i] for chrom_pc, i in zip(*hic_pc)])
x, y = neither_nan(x,y) 
plt.hexbin(x, y, gridsize=20)
sm = np.logical_or(np.logical_and(x > 0, y > 0),np.logical_and(x < 0, y < 0)).sum()
print 'Total 1Mb bins: {0}'.format(len(x))
print '1Mb bins with same compartment in both Dixon-HiC and combined GAM data: {0}'.format(sm)
print '% 1Mb bins with shared compartment: {0:.1%}'.format(float(sm) / len(x))
print 'Pearson corr = {}'.format(pearsonr(x, y)[0])
print 'median x:', np.median(x)
print pd.crosstab((x>0),(y>0))

plt.xlabel('Combined GAM PCA')
plt.ylabel('Dixon Hi-C PCA')

In [ ]:
plt.hist(x, bins=30)